**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
which bedtools
bedtools -version

/nfs/software/helmod/apps/Core/bedtools2/2.25.0-fasrc01/bin/bedtools
bedtools v2.25.0


**Check data directory**

In [3]:
ls ${FD_RES}/A001_K562_WSTARRseq/fragment/*unstranded.bed | xargs -n 1 basename

A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed
A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed
A001_K562_WSTARRseq_Input_rep2.GATA1.unstranded.bed
A001_K562_WSTARRseq_Input_rep2.MYC.unstranded.bed
A001_K562_WSTARRseq_Input_rep3.GATA1.unstranded.bed
A001_K562_WSTARRseq_Input_rep3.MYC.unstranded.bed
A001_K562_WSTARRseq_Input_rep4.GATA1.unstranded.bed
A001_K562_WSTARRseq_Input_rep4.MYC.unstranded.bed
A001_K562_WSTARRseq_Output_rep1.GATA1.unstranded.bed
A001_K562_WSTARRseq_Output_rep1.MYC.unstranded.bed
A001_K562_WSTARRseq_Output_rep2.GATA1.unstranded.bed
A001_K562_WSTARRseq_Output_rep2.MYC.unstranded.bed
A001_K562_WSTARRseq_Output_rep3.GATA1.unstranded.bed
A001_K562_WSTARRseq_Output_rep3.MYC.unstranded.bed


In [7]:
FDIRY=${FD_RES}/A001_K562_WSTARRseq/fragment
ls ${FDIRY}/*GATA1.unstranded.bed | xargs -n 1 basename | wc -l
ls ${FDIRY}/*MYC.unstranded.bed   | xargs -n 1 basename | wc -l

7
7


**Check chromosome**

In [8]:
FDIRY=${FD_RES}/A001_K562_WSTARRseq/fragment

FNAME=A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed
head -1 ${FDIRY}/${FNAME}

FNAME=A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed
head -1 ${FDIRY}/${FNAME}

chrX	47787533	47787772	chrX_47787533_47787772	1	.
chr8	126736069	126736518	chr8_126736069_126736518	1	.


```
GATA1 --- chrX
MYC   --- chr8
```

## Calculate NUC for fragments in Whole-Genome-STARR

### Region: GATA

**RUN**

In [11]:
sbatch -p ${NODE} \
    -J bedtools_nuc_wstarr_gata1 \
    --mem 20G \
    --array 0-6 \
    -o ${FD_LOG}/fragment_nuc_wstarr_gata1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
CHROM=chrX
REGION=GATA1

### set input and output
FD_BED=${FD_RES}/A001_K562_WSTARRseq/fragment
FP_BEDS=($(ls ${FD_BED}/*${REGION}.unstranded.bed))
FP_BED=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=$(basename ${FP_BED})

FD_OUT=${FD_RES}/A001_K562_WSTARRseq/fragment_nuc
FN_OUT=${FN_BED}.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show input/output file
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}

### calculate the GC content of each fragment
bedtools nuc \
    -fi  ${FD_GEN}/${FN_GEN} \
    -bed ${FD_BED}/${FN_BED} |\
    gzip -c > ${FD_OUT}/${FN_OUT}

### Show input/output file
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29147828


**CHECK**

In [12]:
cat ${FD_LOG}/fragment_nuc_wstarr_gata1.0.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-11-22+21:50:47

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment/A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment_nuc/A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment/A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed
chrX	47787533	47787772	chrX_47787533_47787772	1	.
chrX	47787569	47787735	chrX_47787569_47787735	1	.
chrX	47787714	47788125	chrX_47787714_47788125	1	.

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment_nuc/A001_K562_WSTARRseq_Input_rep1.GATA1.unstranded.bed.gz
#1_usercol	2_usercol	3_usercol	4_usercol	5_usercol	6

### Region: MYC

**RUN**

In [13]:
sbatch -p ${NODE} \
    -J bedtools_nuc_wstarr_myc \
    --mem 20G \
    --array 0-6 \
    -o ${FD_LOG}/fragment_nuc_wstarr_myc.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
CHROM=chr8
REGION=MYC

### set input and output
FD_BED=${FD_RES}/A001_K562_WSTARRseq/fragment
FP_BEDS=($(ls ${FD_BED}/*${REGION}.unstranded.bed))
FP_BED=${FP_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=$(basename ${FP_BED})

FD_OUT=${FD_RES}/A001_K562_WSTARRseq/fragment_nuc
FN_OUT=${FN_BED}.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show input/output file
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}

### calculate the GC content of each fragment
bedtools nuc \
    -fi  ${FD_GEN}/${FN_GEN} \
    -bed ${FD_BED}/${FN_BED} |\
    gzip -c > ${FD_OUT}/${FN_OUT}

### Show input/output file
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29147835


**CHECK**

In [14]:
cat ${FD_LOG}/fragment_nuc_wstarr_myc.0.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         10-11-22+21:51:17

Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment/A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment_nuc/A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment/A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed
chr8	126736069	126736518	chr8_126736069_126736518	1	.
chr8	126736096	126736559	chr8_126736096_126736559	1	.
chr8	126736430	126736781	chr8_126736430_126736781	1	.

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/A001_K562_WSTARRseq/fragment_nuc/A001_K562_WSTARRseq_Input_rep1.MYC.unstranded.bed.gz
#1_usercol	2_usercol	3_usercol	4_usercol	5_userc